<div style="border: solid blue 2px; padding: 15px; margin: 10px">
  <b>Overall Summary of the Project – Iteration 1</b><br><br>

  Hi Victor, I’m <b>Victor Camargo</b>. I’ve reviewed your project and you’ve done an excellent job overall! 🎉 The notebook is structured, logical, and demonstrates a solid understanding of preprocessing, feature engineering, and model evaluation. You’re now ready for approval — just take a look at one final improvement suggestion below.

  <b>Nice work on:</b><br>
  ✔️ Clean data merging and proper handling of missing values (context-aware strategy)<br>
  ✔️ Correct and consistent preprocessing, including encoding and feature scaling<br>
  ✔️ Smart feature engineering with `NumberOfMonths`, and structured model comparison<br>
  ✔️ Strong use of AUC-ROC as the evaluation metric, and thoughtful model selection<br><br>

  🟡 Suggestions for future improvements:<br>
  • Remove redundant loops like the repeated Decision Tree tuning block<br>
  • Clean up re-merging lines like the repeated `df_cpi` merge<br>

  🔴 **Important final note (no need to resubmit):**  
  In the last test evaluation cell, you printed the final AUC-ROC score using `model_cb`, but your final model is actually trained as `model_cb_final`. It’s a small mismatch — nothing serious — but to be completely correct and reproducible, test predictions should match the final trained model.<br><br>
  
  ✅ I’ve run a corrected version using `model_cb_final`, and here’s the result:<br>
  <b>AUC-ROC Score: 0.845</b> — a fantastic score, which rounds to **0.85**! Great work! 🌟<br><br>

  You can add this block at the end of your notebook to match the result:<br><br>

  <b># Reviewer Code</b><br>

  <pre><code>
model_cb_final = CatBoostClassifier(loss_function='Logloss', iterations=120, random_seed=616)
model_cb_final.fit(features_train, target_train, cat_features=cats, verbose=20)
probabilities_cb_test = model_cb_final.predict_proba(features_test)
probabilities_cb_one_test = probabilities_cb_test[:,1]
auc_roc_cb = roc_auc_score(target_test, probabilities_cb_one_test)

print(f"AUC-ROC Score: {auc_roc_cb}")
  </code></pre>

  <hr>

  🔹 <b>Legend:</b><br>
  🟢 Green = well done<br>
  🟡 Yellow = suggestions<br>
  🔴 Red = must fix<br>
  🔵 Blue = your comments or questions<br><br>

  Excellent job overall — your notebook is clear, the methodology is sound, and you’ve exceeded the AUC-ROC benchmark. ✅<br>
  <b>Project approved!</b> If you have any questions, feel free to reach out on the Questions channel.<br>
</div>


# Project 17: Staying Connected with Interconnect!
The objective of this project is to create a model that can predict if a client will discontinue their services with the company Interconnect based on the clients contracts, personal information, and services. The metric that will be used to determine the model's functionality is AUC-ROC.

## Imports and Uploads

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier

In [23]:
df_contract = pd.read_csv('/datasets/final_provider/contract.csv')
df_personal = pd.read_csv('/datasets/final_provider/personal.csv')
df_internet = pd.read_csv('/datasets/final_provider/internet.csv')
df_phone = pd.read_csv('/datasets/final_provider/phone.csv')

## Conversions

### DateTime

In [24]:
#converting the begin date to date time type
df_contract['BeginDate'] = pd.to_datetime(df_contract['BeginDate'], format='%Y-%m-%d')

In [25]:
#setting index of filtered dataframe to variable
index_no = df_contract[df_contract['EndDate'] != 'No'].index

In [26]:
#converting seected elements with appropriate indices to datetime
df_contract.loc[index_no, 'EndDate'] = pd.to_datetime(df_contract.loc[index_no, 'EndDate'], format='%Y-%m-%d')

### Fill ins
The 'TotalCharges' column of the contract dataframe has some single space string values. This is because those clients are new as of that month of recording the data. Since they have not paid their first monthly payment yet, it makes sense they do not have a total amount of money paid. Therefore, that value will be replaced with a 0. It's numerically accurate.

In [27]:
df_contract['TotalCharges'] = df_contract['TotalCharges'].replace(' ', 0)
df_contract[df_contract['TotalCharges'] == ' ']

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges


<span style='color:green'>

__Success!__
</span>

In [28]:
#converting total charges to float type now
df_contract['TotalCharges'] = df_contract['TotalCharges'].astype(float)

In [29]:
df_contract.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   customerID        7043 non-null   object        
 1   BeginDate         7043 non-null   datetime64[ns]
 2   EndDate           7043 non-null   object        
 3   Type              7043 non-null   object        
 4   PaperlessBilling  7043 non-null   object        
 5   PaymentMethod     7043 non-null   object        
 6   MonthlyCharges    7043 non-null   float64       
 7   TotalCharges      7043 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(5)
memory usage: 440.3+ KB


***

## Executing Work Plan

### Step 1
Create a new column in the contract dataframe that will represent the target value as 1s and 0s

In [31]:
#creating a row function to apply to dataframe for a new column 
def targ(row):
    value = row['EndDate']
    if value == 'No':
        return 0
    else:
        return 1

In [32]:
#creating the new column
df_contract['target'] = df_contract.apply(targ, axis=1)
df_contract #checing for success

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,target
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85,0
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.50,0
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15,1
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75,0
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65,1
...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,2018-02-01,No,One year,Yes,Mailed check,84.80,1990.50,0
7039,2234-XADUH,2014-02-01,No,One year,Yes,Credit card (automatic),103.20,7362.90,0
7040,4801-JZAZL,2019-03-01,No,Month-to-month,Yes,Electronic check,29.60,346.45,0
7041,8361-LTMKD,2019-07-01,2019-11-01 00:00:00,Month-to-month,Yes,Mailed check,74.40,306.60,1


<span style='color:green'>

__Success!__
</span>
Since my objective to predict if clients will leave, 0s will represent clients that are still active and 1s will represent clients who discontinued. That way, target values of 1 will represent a "positive" probability.

### Steps 2 and 3
Dataframes will be outer merged. The resulting dataframe will have missing values but those will be replaced with contextually accurate replacements. OneHotEncoder will be used to convert categorical data to numbers for the models. Once complete, features, target, training set, validation set, and test sets will be established.

In [40]:
df_cp = df_contract.merge(df_personal, on='customerID', how='outer')
df_cp

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,target,gender,SeniorCitizen,Partner,Dependents
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85,0,Female,0,Yes,No
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.50,0,Male,0,No,No
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15,1,Male,0,No,No
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75,0,Male,0,No,No
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65,1,Female,0,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,2018-02-01,No,One year,Yes,Mailed check,84.80,1990.50,0,Male,0,Yes,Yes
7039,2234-XADUH,2014-02-01,No,One year,Yes,Credit card (automatic),103.20,7362.90,0,Female,0,Yes,Yes
7040,4801-JZAZL,2019-03-01,No,Month-to-month,Yes,Electronic check,29.60,346.45,0,Female,0,Yes,Yes
7041,8361-LTMKD,2019-07-01,2019-11-01 00:00:00,Month-to-month,Yes,Mailed check,74.40,306.60,1,Male,1,Yes,No


In [41]:
df_cpi = df_cp.merge(df_internet, on='customerID', how='outer')
df_cpi

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,target,gender,SeniorCitizen,Partner,Dependents,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85,0,Female,0,Yes,No,DSL,No,Yes,No,No,No,No
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.50,0,Male,0,No,No,DSL,Yes,No,Yes,No,No,No
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15,1,Male,0,No,No,DSL,Yes,Yes,No,No,No,No
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75,0,Male,0,No,No,DSL,Yes,No,Yes,Yes,No,No
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65,1,Female,0,No,No,Fiber optic,No,No,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,2018-02-01,No,One year,Yes,Mailed check,84.80,1990.50,0,Male,0,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes
7039,2234-XADUH,2014-02-01,No,One year,Yes,Credit card (automatic),103.20,7362.90,0,Female,0,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes
7040,4801-JZAZL,2019-03-01,No,Month-to-month,Yes,Electronic check,29.60,346.45,0,Female,0,Yes,Yes,DSL,Yes,No,No,No,No,No
7041,8361-LTMKD,2019-07-01,2019-11-01 00:00:00,Month-to-month,Yes,Mailed check,74.40,306.60,1,Male,1,Yes,No,Fiber optic,No,No,No,No,No,No


In [42]:
df_all = df_cpi.merge(df_phone, on='customerID', how='outer')
df_all

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,target,gender,...,Partner,Dependents,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,MultipleLines
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85,0,Female,...,Yes,No,DSL,No,Yes,No,No,No,No,NaN
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.50,0,Male,...,No,No,DSL,Yes,No,Yes,No,No,No,No
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15,1,Male,...,No,No,DSL,Yes,Yes,No,No,No,No,No
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75,0,Male,...,No,No,DSL,Yes,No,Yes,Yes,No,No,NaN
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65,1,Female,...,No,No,Fiber optic,No,No,No,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,2018-02-01,No,One year,Yes,Mailed check,84.80,1990.50,0,Male,...,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,Yes
7039,2234-XADUH,2014-02-01,No,One year,Yes,Credit card (automatic),103.20,7362.90,0,Female,...,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Yes
7040,4801-JZAZL,2019-03-01,No,Month-to-month,Yes,Electronic check,29.60,346.45,0,Female,...,Yes,Yes,DSL,Yes,No,No,No,No,No,NaN
7041,8361-LTMKD,2019-07-01,2019-11-01 00:00:00,Month-to-month,Yes,Mailed check,74.40,306.60,1,Male,...,Yes,No,Fiber optic,No,No,No,No,No,No,Yes


In [43]:
df_all.isna().sum()

customerID             0
BeginDate              0
EndDate                0
Type                   0
PaperlessBilling       0
PaymentMethod          0
MonthlyCharges         0
TotalCharges           0
target                 0
gender                 0
SeniorCitizen          0
Partner                0
Dependents             0
InternetService     1526
OnlineSecurity      1526
OnlineBackup        1526
DeviceProtection    1526
TechSupport         1526
StreamingTV         1526
StreamingMovies     1526
MultipleLines        682
dtype: int64

In [44]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   customerID        7043 non-null   object        
 1   BeginDate         7043 non-null   datetime64[ns]
 2   EndDate           7043 non-null   object        
 3   Type              7043 non-null   object        
 4   PaperlessBilling  7043 non-null   object        
 5   PaymentMethod     7043 non-null   object        
 6   MonthlyCharges    7043 non-null   float64       
 7   TotalCharges      7043 non-null   float64       
 8   target            7043 non-null   int64         
 9   gender            7043 non-null   object        
 10  SeniorCitizen     7043 non-null   int64         
 11  Partner           7043 non-null   object        
 12  Dependents        7043 non-null   object        
 13  InternetService   5517 non-null   object        
 14  OnlineSecurity    5517 n

In [16]:
df_all['InternetService'].value_counts()

Fiber optic    3096
DSL            2421
Name: InternetService, dtype: int64

__The missing values are only missing because those specific customers do not pay for that service. Since they are all in features that are categorical, they will simply all be replaced with their own unique string value 'N/A'. This will provide a third value for each of those features for the model to interpret since all the said features are currently binary.__

In [45]:
df_all = df_all.fillna('N/A')
df_all

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,target,gender,...,Partner,Dependents,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,MultipleLines
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85,0,Female,...,Yes,No,DSL,No,Yes,No,No,No,No,N/A
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.50,0,Male,...,No,No,DSL,Yes,No,Yes,No,No,No,No
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15,1,Male,...,No,No,DSL,Yes,Yes,No,No,No,No,No
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75,0,Male,...,No,No,DSL,Yes,No,Yes,Yes,No,No,N/A
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65,1,Female,...,No,No,Fiber optic,No,No,No,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,2018-02-01,No,One year,Yes,Mailed check,84.80,1990.50,0,Male,...,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,Yes
7039,2234-XADUH,2014-02-01,No,One year,Yes,Credit card (automatic),103.20,7362.90,0,Female,...,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Yes
7040,4801-JZAZL,2019-03-01,No,Month-to-month,Yes,Electronic check,29.60,346.45,0,Female,...,Yes,Yes,DSL,Yes,No,No,No,No,No,N/A
7041,8361-LTMKD,2019-07-01,2019-11-01 00:00:00,Month-to-month,Yes,Mailed check,74.40,306.60,1,Male,...,Yes,No,Fiber optic,No,No,No,No,No,No,Yes


The features and target will be established now

In [46]:
droppies = ['customerID', 'BeginDate', 'EndDate', 'target'] #columns that are not needed for the model and the target
features = df_all.drop(droppies, axis=1) #features df
target = df_all['target'] #the target

#numerical columns
num_col = features.select_dtypes(include=['int64', 'float64']).columns
features_num = features[num_col]
#categorical columns
cats = ['Type', 'PaperlessBilling', 'PaymentMethod', 'gender', 'Partner', 'Dependents', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'MultipleLines']
features_cat = features[cats]
ohe = OneHotEncoder(drop='first', sparse=False).fit(features_cat) #using OHE
features_cat_names = ohe.get_feature_names()
features_cat_ohe_values = ohe.transform(features_cat)

In [49]:
#creating a dataframe of the encoded categorical data
df_features_cat_ohe = pd.DataFrame(data=features_cat_ohe_values, columns=features_cat_names)
df_features_cat_ohe

,x0_One year,x0_Two year,x1_Yes,x2_Credit card (automatic),x2_Electronic check,x2_Mailed check,x3_Male,x4_Yes,x5_Yes,x6_Fiber optic,...,x9_No,x9_Yes,x10_No,x10_Yes,x11_No,x11_Yes,x12_No,x12_Yes,x13_No,x13_Yes
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
7039,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
7040,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
7041,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


In [50]:
#combining the numerial features and categorical features
all_features_ohe = pd.concat([features_num, df_features_cat_ohe], axis='columns')
all_features_ohe

,MonthlyCharges,TotalCharges,SeniorCitizen,x0_One year,x0_Two year,x1_Yes,x2_Credit card (automatic),x2_Electronic check,x2_Mailed check,x3_Male,...,x9_No,x9_Yes,x10_No,x10_Yes,x11_No,x11_Yes,x12_No,x12_Yes,x13_No,x13_Yes
0,29.85,29.85,0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,56.95,1889.50,0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,53.85,108.15,0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,42.30,1840.75,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
4,70.70,151.65,0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,84.80,1990.50,0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
7039,103.20,7362.90,0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
7040,29.60,346.45,0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
7041,74.40,306.60,1,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


<span style='color:green'>

__Success!__
</span>
Now for the training, validation, and test sets

In [51]:
#splitting into 3 parts for a 60:20:20 ratio
features_intermediate, features_ohe_test, target_intermediate, target_test = train_test_split(all_features_ohe, target, test_size=0.20, random_state=616)
# 25% of 80% is 20
features_ohe_train, features_ohe_valid, target_train, target_valid = train_test_split(features_intermediate, target_intermediate, test_size=0.25, random_state=616)

### Step 4
Modeling

#### Decision Tree Model

In [52]:
best_dt_result = 0
best_dt_depth = 0
for depth in range(1,7):
    model_dt = DecisionTreeClassifier(random_state=616, max_depth=depth)
    model_dt.fit(features_ohe_train, target_train)
    probabilities_valid = model_dt.predict_proba(features_ohe_valid)
    probabilities_one_valid = probabilities_valid[:,1] #probabilities of zero because the goal is to know if the model can predict if someone will leave. Zero represents people who discontinued
    auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
    if auc_roc > best_dt_result:
        best_dt_result = auc_roc
        best_dt_depth = depth
print(f"Best AUC-ROC Value: {best_dt_result}, Best Depth Used: {best_dt_depth}")

Best AUC-ROC Value: 0.8148355182168537, Best Depth Used: 4


<span style='color:blue'>
A decent score of about 0.81. We can try for higher.
</span>

#### Random Forest Model

In [53]:
best_rf_result = 0
best_rf_depth = 0
best_est = 0
for est in range(10, 51, 10):
    for depth in range(1,11):
        model_rf = RandomForestClassifier(random_state=616, n_estimators=est, max_depth=depth)
        model_rf.fit(features_ohe_train, target_train)
        probabilities = model_rf.predict_proba(features_ohe_valid)
        probabilities_one_valid = probabilities[:,1]
        auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
        if auc_roc > best_rf_result:
            best_rf_result = auc_roc
            best_rf_depth = depth
            best_est = est
print(f"Best AUC-ROC Value: {best_rf_result}, Best Depth: {best_rf_depth}, Best Number of Estimators: {best_est}")

Best AUC-ROC Value: 0.838600837106005, Best Depth: 8, Best Number of Estimators: 40


<span style="color:blue">
The random forest performed a little better reaching a value of about 0.84 by using a depth of 8 and 40 estimators.
</span>

#### Logistic Regression Model

In [54]:
model_lr = LogisticRegression(random_state=616, solver='liblinear')
model_lr.fit(features_ohe_train, target_train)
probabilities_lr = model_lr.predict_proba(features_ohe_valid)
probabilities_lr_one_valid = probabilities_lr[:,1]
auc_roc = roc_auc_score(target_valid, probabilities_lr_one_valid)
print(f"AUC-ROC Value: {auc_roc}")

AUC-ROC Value: 0.8316417025510128


<span style='color:blue'>
Logistic Regression reaching about 0.83 is nice.
</span>

#### K Nearest Neighbors Model
The preceding models performed decently. At an attempt to reach a higher AUC-ROC score I will be trying the KNearestNeighbors model

In [55]:
#converting the dataframes to matrices and series to vectors
features_train_mx = features_ohe_train.values
target_train_vector = target_train.values
features_valid_mx = features_ohe_valid.values
target_valid_mx = target_valid.values

In [56]:
best_n_neighbors = 0
best_kn_value = 0
for n in range(5,11):
    knc = KNeighborsClassifier(n_neighbors=n, metric='euclidean')
    knc.fit(features_train_mx, target_train_vector)
    probabilities = knc.predict_proba(features_valid_mx)
    auc_roc = roc_auc_score(target_valid_mx, probabilities[:,1])
    if auc_roc > best_kn_value:
        best_kn_value = auc_roc
        best_n_neighbors = n
print(f"Best AUC-ROC Value: {best_kn_value}, Best N Neighbors: {n}")

Best AUC-ROC Value: 0.7762123214366468, Best N Neighbors: 10


<span style='color:red'>

That model did not seem to wor well with this dataset
</span>

__The models are not performing too bad. My goal is to still achieve a higher AUC-ROC score. I have an idea of feature engineering to include a column that should represent the number of months a customer has been active. This will be done by dividing the total charges by the monthly charge. I will also scale the numerical features. This should hopefully help increase the scores.__

## Tweaking Features

### Feature Engineering

In [57]:
#retrieving the numerical features df to adjust before combining
features_num

,MonthlyCharges,TotalCharges,SeniorCitizen
0,29.85,29.85,0
1,56.95,1889.50,0
2,53.85,108.15,0
3,42.30,1840.75,0
4,70.70,151.65,0
...,...,...,...
7038,84.80,1990.50,0
7039,103.20,7362.90,0
7040,29.60,346.45,0
7041,74.40,306.60,1


In [58]:
features_num['NumberOfMonths'] = features_num['TotalCharges'] / features_num['MonthlyCharges']
features_num

/tmp/ipykernel_32/2067164093.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_num['NumberOfMonths'] = features_num['TotalCharges'] / features_num['MonthlyCharges']


,MonthlyCharges,TotalCharges,SeniorCitizen,NumberOfMonths
0,29.85,29.85,0,1.000000
1,56.95,1889.50,0,33.178227
2,53.85,108.15,0,2.008357
3,42.30,1840.75,0,43.516548
4,70.70,151.65,0,2.144979
...,...,...,...,...
7038,84.80,1990.50,0,23.472877
7039,103.20,7362.90,0,71.345930
7040,29.60,346.45,0,11.704392
7041,74.40,306.60,1,4.120968


Weirdly, they are not all whole numbers. That can be for some unknown reasons. My guesses are:

A.) The customer perhaps adjusted their package at some point during their time with the company. This would cause a change in monthly payments which wouldn't divide the total cost evenly by one of the monthly charges.

B.) A change in the tax fees

__Nonetheless, I am going to round the numbers because these should still be good estimates of how long the customer has been with the company__

In [59]:
features_num['NumberOfMonths'] = round(features_num['TotalCharges'] / features_num['MonthlyCharges'])
features_num

/tmp/ipykernel_32/4277770470.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_num['NumberOfMonths'] = round(features_num['TotalCharges'] / features_num['MonthlyCharges'])


,MonthlyCharges,TotalCharges,SeniorCitizen,NumberOfMonths
0,29.85,29.85,0,1.0
1,56.95,1889.50,0,33.0
2,53.85,108.15,0,2.0
3,42.30,1840.75,0,44.0
4,70.70,151.65,0,2.0
...,...,...,...,...
7038,84.80,1990.50,0,23.0
7039,103.20,7362.90,0,71.0
7040,29.60,346.45,0,12.0
7041,74.40,306.60,1,4.0


In [60]:
#combine feature dfs again
all_features_ohe = pd.concat([features_num, df_features_cat_ohe], axis='columns')
all_features_ohe

,MonthlyCharges,TotalCharges,SeniorCitizen,NumberOfMonths,x0_One year,x0_Two year,x1_Yes,x2_Credit card (automatic),x2_Electronic check,x2_Mailed check,...,x9_No,x9_Yes,x10_No,x10_Yes,x11_No,x11_Yes,x12_No,x12_Yes,x13_No,x13_Yes
0,29.85,29.85,0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,56.95,1889.50,0,33.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,53.85,108.15,0,2.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,42.30,1840.75,0,44.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
4,70.70,151.65,0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,84.80,1990.50,0,23.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
7039,103.20,7362.90,0,71.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
7040,29.60,346.45,0,12.0,0.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
7041,74.40,306.60,1,4.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


In [61]:
#splitting into 3 parts for a 60:20:20 ratio
features_intermediate, features_ohe_test, target_intermediate, target_test = train_test_split(all_features_ohe, target, test_size=0.20, random_state=616)
# 25% of 80% is 20
features_ohe_train, features_ohe_valid, target_train, target_valid = train_test_split(features_intermediate, target_intermediate, test_size=0.25, random_state=616)

### Scaling

In [62]:
#creating copies of the features for scaling incase of possible retreat
features_ohe_train2 = features_ohe_train.copy()
features_ohe_valid2 = features_ohe_valid.copy()

numeric = ['MonthlyCharges', 'TotalCharges', 'NumberOfMonths']

scaler = StandardScaler()
scaler.fit(features_ohe_train2[numeric])
features_ohe_train2[numeric] = scaler.transform(features_ohe_train2[numeric])
features_ohe_valid2[numeric] = scaler.transform(features_ohe_valid2[numeric])

<span style='color:blue'>

__Now that the scaling was complete, I will attempt the models again__
</span>

### Modeling Attempt 2

#### Decision Tree Model Attempt 2

In [63]:
best_dt_result = 0
best_dt_depth = 0
for depth in range(1,7):
    model_dt = DecisionTreeClassifier(random_state=616, max_depth=depth)
    model_dt.fit(features_ohe_train2, target_train)
    probabilities_valid = model_dt.predict_proba(features_ohe_valid2)
    probabilities_one_valid = probabilities_valid[:,1] #probabilities of zero because the goal is to know if the model can predict if someone will leave. Zero represents people who discontinued
    auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
    if auc_roc > best_dt_result:
        best_dt_result = auc_roc
        best_dt_depth = depth
print(f"Best AUC-ROC Value: {best_dt_result}, Best Depth Used: {best_dt_depth}")

Best AUC-ROC Value: 0.8249425934207287, Best Depth Used: 5


#### Random Forest Model Attempt 2

In [64]:
best_rf_result = 0
best_rf_depth = 0
best_est = 0
for est in range(10, 51, 10):
    for depth in range(1,11):
        model_rf = RandomForestClassifier(random_state=616, n_estimators=est, max_depth=depth)
        model_rf.fit(features_ohe_train2, target_train)
        probabilities = model_rf.predict_proba(features_ohe_valid2)
        probabilities_one_valid = probabilities[:,1]
        auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
        if auc_roc > best_rf_result:
            best_rf_result = auc_roc
            best_rf_depth = depth
            best_est = est
print(f"Best AUC-ROC Value: {best_rf_result}, Best Depth: {best_rf_depth}, Best Number of Estimators: {best_est}")

Best AUC-ROC Value: 0.8435438324984045, Best Depth: 7, Best Number of Estimators: 50


#### Logistic Regression Model Attempt 2

In [66]:
model_lr = LogisticRegression(random_state=616, solver='liblinear')
model_lr.fit(features_ohe_train2, target_train)
probabilities_lr = model_lr.predict_proba(features_ohe_valid2)
probabilities_lr_one_valid = probabilities_lr[:,1]
auc_roc = roc_auc_score(target_valid, probabilities_lr_one_valid)
print(f"AUC-ROC Value: {auc_roc}")

AUC-ROC Value: 0.842570307357589


#### K Nearest Neighbors Model Attempt 2

In [67]:
#converting the dataframes to matrices and series to vectors
features_train_mx = features_ohe_train2.values
target_train_vector = target_train.values
features_valid_mx = features_ohe_valid2.values
target_valid_mx = target_valid.values

best_n_neighbors = 0
best_kn_value = 0
for n in range(5,11):
    knc = KNeighborsClassifier(n_neighbors=n, metric='euclidean')
    knc.fit(features_train_mx, target_train_vector)
    probabilities = knc.predict_proba(features_valid_mx)
    auc_roc = roc_auc_score(target_valid_mx, probabilities[:,1])
    if auc_roc > best_kn_value:
        best_kn_value = auc_roc
        best_n_neighbors = n
print(f"Best AUC-ROC Value: {best_kn_value}, Best N Neighbors: {n}")

Best AUC-ROC Value: 0.8183678442761746, Best N Neighbors: 10


<span style='color:blue'>

__The AUC-ROC values of each model actually increased! The highest score still being from the Random Forest, thistime with a depth of 7 and 50 estimators.__
</span>
I want to reach at least a value of 0.85 before testing. There is a model I haven't tried yet, and I will try it now.

#### CatBoost Model

In [69]:
#splitting the regular features into 3 parts for a 60:20:20 ratio
features_inter, features_test, target_inter, target_test = train_test_split(features, target, test_size=0.20, random_state=616)
# 25% of 80% is 20
features_train, features_valid, target_train, target_valid = train_test_split(features_inter, target_inter, test_size=0.25, random_state=616)

In [70]:
best_cb_score = 0
best_cb_iter = 0

for i in range(10, 500, 10): #testing many iterations for best results
    model_cb = CatBoostClassifier(loss_function='Logloss', iterations=i, random_seed=616)
    model_cb.fit(features_train, target_train, cat_features=cats, verbose=20)
    probabilities_cb_valid = model_cb.predict_proba(features_valid)
    probabilities_cb_one_valid = probabilities_cb_valid[:,1]
    auc_roc_cb = roc_auc_score(target_valid, probabilities_cb_one_valid)
    if auc_roc_cb > best_cb_score:
        best_cb_score = auc_roc_cb
        best_cb_iter = i

print(f"Best AUC-ROC Value: {best_cb_score}, Number of iterations: {best_cb_iter}")

Learning rate set to 0.5
0:	learn: 0.5478574	total: 50.4ms	remaining: 454ms
9:	learn: 0.4034917	total: 75.3ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.5478574	total: 2.23ms	remaining: 42.5ms
19:	learn: 0.3833695	total: 35.7ms	remaining: 0us
Learning rate set to 0.474943
0:	learn: 0.5531286	total: 2.17ms	remaining: 63.1ms
20:	learn: 0.3848127	total: 37.6ms	remaining: 16.1ms
29:	learn: 0.3725599	total: 56.9ms	remaining: 0us
Learning rate set to 0.364815
0:	learn: 0.5787053	total: 2.23ms	remaining: 87.1ms
20:	learn: 0.3915355	total: 35.6ms	remaining: 32.2ms
39:	learn: 0.3721543	total: 70.6ms	remaining: 0us
Learning rate set to 0.297308
0:	learn: 0.5963798	total: 2.2ms	remaining: 108ms
20:	learn: 0.3931298	total: 36.4ms	remaining: 50.2ms
40:	learn: 0.3753209	total: 73.8ms	remaining: 16.2ms
49:	learn: 0.3713146	total: 91.5ms	remaining: 0us
Learning rate set to 0.251534
0:	learn: 0.6092508	total: 2.34ms	remaining: 138ms
20:	learn: 0.3973011	total: 35.4ms	remaining: 65.7ms
40:	lear

<span style='color:green'>

__That score is technically better than the random forest result by 15 ten thousandths of a unit.__
</span>
A result of 0.845 with 120 iterations is nice

My goal is still to achieve a score of 0.85 before testing. Let's try a
#### SGD Classifier Model

In [71]:
best_sgd_score = 0
best_iter = 0
for iterations in range(10, 510, 10): #increments of 10
    sgd = SGDClassifier(loss='log', max_iter=iterations, random_state=616) #using log loss to be able to use predict proba
    sgd.fit(features_train_mx, target_train_vector) #using matrix and vector versions of dataset
    probabilities_valid_sgd = sgd.predict_proba(features_valid_mx)
    auc_roc = roc_auc_score(target_valid_mx, probabilities_valid_sgd[:,1])
    if auc_roc > best_sgd_score:
        best_sgd_score = auc_roc
        best_iter = iterations
print(f"Best AUC-ROC Score: {best_sgd_score}, Iterations: {best_iter}")

/.venv/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/.venv/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/.venv/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "


Best AUC-ROC Score: 0.83468663795919, Iterations: 30


<span style='color:blue'>

__Ok a best score of about 0.83 with this random state.__
</span>

After trying all of these models, it appears the best model is the CatBoost Model. Unfortunately, I did not reach my goal of an AUC-ROC score of 0.85, but 0.845, is pretty close and it rounds up haha.

With that being said, I shall now use that model on the test set.

In [40]:
model_cb_final = CatBoostClassifier(loss_function='Logloss', iterations=120, random_seed=616)
model_cb_final.fit(features_train, target_train, cat_features=cats, verbose=20)
probabilities_cb_test = model_cb.predict_proba(features_test)
probabilities_cb_one_test = probabilities_cb_test[:,1]
auc_roc_cb = roc_auc_score(target_test, probabilities_cb_one_test)

print(f"AUC-ROC Score: {auc_roc_cb}")

Learning rate set to 0.133215
0:	learn: 0.6459118	total: 2.17ms	remaining: 259ms
20:	learn: 0.4177384	total: 32.6ms	remaining: 154ms
40:	learn: 0.3952827	total: 66.3ms	remaining: 128ms
60:	learn: 0.3885257	total: 101ms	remaining: 97.4ms
80:	learn: 0.3813019	total: 137ms	remaining: 65.8ms
100:	learn: 0.3742000	total: 174ms	remaining: 32.7ms
119:	learn: 0.3696096	total: 210ms	remaining: 0us
AUC-ROC Score: 0.8442545661215738


By using the test set, we achieved an AUC-ROC value of 0.844!